In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
from pathlib import Path
import glob
from collections import Counter, defaultdict
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import re
from pathlib import Path

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add src to path
sys.path.append('../src')
from config import config

In [30]:
try:
    listing_df = pd.read_csv(config.listing_path)
    print(f"Listings loaded: {len(listing_df):,} rows")
except FileNotFoundError:
    print("listings.csv not found. Please ensure dataset is in data/ folder")
    listings_df = None

try:
    duplicates_df = pd.read_csv(config.duplicates_path)
    print(f"Duplicates loaded: {len(duplicates_df):,} pairs")
except FileNotFoundError:
    print("duplicate_listing.csv not found. Please ensure dataset is in data/ folder")
    duplicates_df = None

# Check for pictures directory
pictures_path = config.pictures_path
if pictures_path.exists():
    image_files = list(pictures_path.glob("*.*"))
    print(f"Pictures directory found: {len(image_files):,} images")
else:
    print("Pictures directory not found")
    image_files = []

AttributeError: 'ProjectConfig' object has no attribute 'listing_path'